In [407]:
import pandas as pd
from tqdm import tqdm
import json
import numpy as np
import time
from copy import deepcopy
import matplotlib.pyplot as plt
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import SVD, accuracy
from surprise import KNNBasic
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math

In [66]:
df_users = pd.read_csv('users_restaurants_illinois_reduced.csv')
df_business = pd.read_csv('businesses_restaurants_illinois_reduced.csv')
df_ratings = pd.read_csv('ratings_restaurants_illinois_reduced.csv')
df_users = df_users.drop(columns=['Unnamed: 0'])
df_business = df_business.drop(columns=['Unnamed: 0'])
df_ratings = df_ratings.drop(columns=['Unnamed: 0'])

In [67]:
df_ratings_train = df_ratings.loc[df_ratings.date < '2018-01-01']
df_ratings_test = df_ratings.loc[df_ratings.date >= '2018-01-01']

In [68]:
df_ratings_train = df_ratings_train.drop(columns=['date'])
df_ratings_test = df_ratings_test.drop(columns=['date'])
df_ratings_train = df_ratings_train.reset_index()
df_ratings_test = df_ratings_test.reset_index()

# Reference measure: Average Rating

We evaluate the RMSE and MAE when we predict the average rating of the train set for the entire test set. It's a reference measure.

In [70]:
average_train = df_ratings_train.rating.mean()
pred_average = np.repeat(average_train, df_ratings_test.shape[0])

In [71]:
mean_squared_error(df_ratings_test['rating'], pred_average)

2.021967996394059

In [72]:
mean_absolute_error(df_ratings_test['rating'], pred_average)

1.2390468228842726

# Graphs Construction

In [19]:
import networkx as nx

## Users Graph

In [73]:
df_users_graph = df_users[['user_id','friends']]

In [74]:
users = []
for i in range(df_users_graph.shape[0]):
    friends = df_users_graph['friends'][i].split(', ')
    users.append([df_users_graph['user_id'][i], friends])

In [75]:
G_users = nx.Graph()

for i in range(len(users)):
    G_users.add_node(users[i][0])

nl = list(G_users.nodes())
for i in range(len(users)):
    user = users[i][0]
    friends = users[i][1]
    for friend in friends:
        if friend in nl:
            G_users.add_edge(user, friend)


## User-Restaurant Bipartite Graph

In [76]:
users_list = df_users['user_id'].tolist()
restaurants_list = df_business['business_id'].tolist()

links_list = []
for i in range(df_ratings_train.shape[0]):
    links_list.append((df_ratings_train['user_id'][i], df_ratings_train['business_id'][i], df_ratings_train['rating'][i]))
    
    

In [77]:
G_us_re = nx.Graph()

G_us_re.add_nodes_from(users_list, bipartite=0)
G_us_re.add_nodes_from(restaurants_list, bipartite=1)

G_us_re.add_weighted_edges_from(links_list)

In [78]:
nx.is_connected(G_us_re)

False

In [79]:
if nx.is_connected(G_us_re):
    restaurants_nodes, users_nodes = nx.bipartite.sets(G_us_re)
else:
    users_nodes = {n for n, d in G_us_re.nodes(data=True) if d['bipartite']==0}
    restaurants_nodes = set(G_us_re) - users_nodes

# Recommendations

## Rating prediction based on User-Restaurant Bipartite Graph

The library Surprise allow to build recommendation systems thanks to collaborative filtering, but it does not support large datasets and leads to kernel crashes. We implemented the methods by our own.

In [80]:
def similarity(u,v,other_nodes, G):
    p, a, b = 0, 0, 0
    for i in other_nodes:
        p += G.get_edge_data(u,i,default={'weight': 0.0})['weight']*G.get_edge_data(v,i,default={'weight': 0.0})['weight']
        a += G.get_edge_data(u,i,default={'weight': 0.0})['weight']**2
        b += G.get_edge_data(v,i,default={'weight': 0.0})['weight']**2
    sim = p/(np.sqrt(a)*np.sqrt(b))
    return sim    

### User Collaborative Filtering

We compute cosine similarities among users.

In [85]:
ar_similarities_user = np.zeros((len(users_list), len(users_list)))
for u in range(len(users_list)):
    for v in range(u, len(users_list)):
        ar_similarities_user[u][v] = similarity(users_list[u],users_list[v],restaurants_nodes,G_us_re)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


In [86]:
ar_similarities_user_f = ar_similarities_user + ar_similarities_user.transpose() - np.eye(len(users_list))
df_similarities_user = pd.DataFrame(ar_similarities_user_f, columns = users_list, index = users_list)

Remark: Some NaN because some users have never published a Rating before the train/test separation date

In [87]:
df_similarities_user_na = df_similarities_user.fillna(0.0)

In [131]:
df_similarities_user.to_csv('similarities_bipartite_users_withnan.csv')

In [132]:
df_similarities_user_na.to_csv('similarities_bipartite_users.csv')

In [121]:
def prediction_userCF(df_similarities,u,i):
    '''
    Returns the predicted rating by user u for restaurant i
    
    Arguments:
        similarities: similarities between users
        u: a user
        i: a restaurant
    '''
    n,d = 0,0
    pred = 0
    for v in users_nodes:
        rating_v_i = G_us_re.get_edge_data(v,i,default={'weight': 0.0})['weight']
        if rating_v_i != 0:
            n += df_similarities[u][v]*rating_v_i
            d += df_similarities[u][v]
    if d !=0:
        pred = n/d
    else:
        pred = average_train
    return pred
    

In [122]:
def make_pred_userCF(df_ratings_test,df_similarities):
    pred = []
    for j in range(df_ratings_test.shape[0]):
        userid = df_ratings_test['user_id'][j]
        businessid = df_ratings_test['business_id'][j]
        pred.append(prediction_userCF(df_similarities,userid,businessid))
    return pred

In [123]:
pred_userCF = make_pred_userCF(df_ratings_test,df_similarities_user_na)

In [124]:
#np.sum(df_similarities_user_na.loc[df_similarities_user_na.index == '4mjnkd8oJVCfBKN3i4rB-g'].values)

In [125]:
mean_squared_error(df_ratings_test['rating'], pred_userCF)

1.9495519710745213

In [126]:
mean_absolute_error(df_ratings_test['rating'], pred_userCF)

1.173219897170781

In [209]:
nb_changed_values = df_ratings_test.shape[0]-pred_userCF.count(average_train)
nb_changed_values

1527

### Item Collaborative Filtering

In [127]:
ar_similarities_rest = np.zeros((len(restaurants_list), len(restaurants_list)))
for u in range(len(restaurants_list)):
    for v in range(u, len(restaurants_list)):
        ar_similarities_rest[u][v] = similarity(restaurants_list[u],restaurants_list[v],users_nodes,G_us_re)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


In [128]:
ar_similarities_rest_f = ar_similarities_rest + ar_similarities_rest.transpose() - np.eye(len(restaurants_list))
df_similarities_rest = pd.DataFrame(ar_similarities_rest_f, columns = restaurants_list, index = restaurants_list)

In [130]:
df_similarities_rest_na = df_similarities_rest.fillna(0.0)

In [235]:
df_similarities_rest.to_csv('similarities_bipartite_rest_withnan.csv')

In [236]:
df_similarities_rest_na.to_csv('similarities_bipartite_rest.csv')

In [138]:
def prediction_itemCF(df_similarities,u,i):
    '''
    Returns the predicted rating by user u for restaurant i
    
    Arguments:
        similarities: similarities between users
        u: a user
        i: a restaurant
    '''
    n,d = 0,0
    pred = 0
    for j in restaurants_nodes:
        rating_u_j = G_us_re.get_edge_data(u,j,default={'weight': 0.0})['weight']
        if rating_u_j != 0:
            n += df_similarities[i][j]*rating_u_j
            d += df_similarities[i][j]
    if d != 0:
        pred = n/d
    else:
        pred = average_train
    return pred


In [139]:
def make_pred_itemCF(df_ratings_test,df_similarities):
    pred = []
    for j in range(df_ratings_test.shape[0]):
        userid = df_ratings_test['user_id'][j]
        businessid = df_ratings_test['business_id'][j]
        pred.append(prediction_itemCF(df_similarities,userid,businessid))
    return pred

In [140]:
pred_itemCF = make_pred_itemCF(df_ratings_test,df_similarities_rest_na)

In [141]:
mean_squared_error(df_ratings_test['rating'], pred_itemCF)

2.1562424689639013

In [142]:
mean_absolute_error(df_ratings_test['rating'], pred_itemCF)

1.1957432506081938

In [210]:
nb_changed_values = df_ratings_test.shape[0]-pred_itemCF.count(average_train)
nb_changed_values

1527

### Latent Collaborative Filtering

In [470]:
reader = Reader(rating_scale = (0.0, 5.0))

train_data = Dataset.load_from_df(df_ratings_train[['user_id', 'business_id', 'rating']], reader)
test_data = Dataset.load_from_df(df_ratings_test[['user_id', 'business_id', 'rating']], reader)

sr_train = train_data.build_full_trainset()
sr_test_before = test_data.build_full_trainset()
sr_test = sr_test_before.build_testset()


In [471]:
algo_latent = SVD()
algo_latent.fit(sr_train)

In [472]:
pred_latent = algo_latent.test(sr_test)

In [487]:
accuracy.mse(pred_latent)

MSE: 1.6826


1.6825817304703514

In [474]:
accuracy.mae(pred_latent)

MAE:  1.0828


1.08284549432576

In [484]:
pred_latent_list = [pred_latent[i][3] for i in range(len(pred_latent))]

In [486]:
nb_changed_values = df_ratings_test.shape[0]-pred_latent_list.count(average_train)
nb_changed_values

3745

In [ ]:
RMSE_tune = {}
n_epochs = [5, 7, 10]
lr_all = [0.002, 0.003, 0.005]
reg_all =  [0.4, 0.5, 0.6]
for n in n_epochs:
    for l in lr_all:
        for r in reg_all:
            algo = SVD(n_epochs = n, lr_all = l, reg_all = r)
            algo.fit(sr_train)
            predictions = algo.test(val_sr)
            RMSE_tune[n,l,r] = accuracy.rmse(predictions)

## Rating prediction based on User-User Graph

#### Jaccard similarity

In [192]:
df_similarities_user_uni = pd.DataFrame(columns = users_list, index = users_list)
for u in users_list:
    for v in users_list:
        jac = nx.jaccard_coefficient(G_users, [(u,v)])
        for a,b,c in jac:
            df_similarities_user_uni[u][v] = c

In [201]:
for u in users_list:
    df_similarities_user_uni[u][u] = 1.0

In [220]:
df_similarities_user_uni.to_csv('similarities_unipartite_jaccard_users.csv')

In [202]:
pred_useruser = make_pred_userCF(df_ratings_test,df_similarities_user_uni)


In [203]:
mean_squared_error(df_ratings_test['rating'], pred_useruser)

2.079411848348647

In [204]:
mean_absolute_error(df_ratings_test['rating'], pred_useruser)

1.2313948392566962

In [208]:
nb_changed_values = df_ratings_test.shape[0]-pred_useruser.count(average_train)
nb_changed_values

674

#### Other similarity measure: FriendTNS

In [232]:
def friendTNS(G_users,u,v):
    sim = 0
    edges = [e for e in G_users.edges()]
    if (u,v) in edges or (v,u) in edges:
        sim = 1/(G_users.degree(u)+G_users.degree(v)-1)
    else:
        sim = 0
    return sim 

In [233]:
ar_similarities_user_uniTNS = np.zeros((len(users_list), len(users_list)))
for u in range(len(users_list)):
    for v in range(u, len(users_list)):
        ar_similarities_user_uniTNS[u][v] = friendTNS(G_users, users_list[u], users_list[v])

In [238]:
ar_similarities_user_uniTNS_f = ar_similarities_user_uniTNS + ar_similarities_user_uniTNS.transpose() - np.eye(len(users_list))
df_similarities_user_uniTNS = pd.DataFrame(ar_similarities_user_uniTNS_f, columns = users_list, index = users_list)

In [239]:
for u in users_list:
    df_similarities_user_uniTNS[u][u] = 1.0

In [241]:
df_similarities_user_uniTNS.to_csv('similarities_unipartite_TNS_users.csv')

In [242]:
pred_useruserTNS = make_pred_userCF(df_ratings_test,df_similarities_user_uniTNS)

In [243]:
mean_squared_error(df_ratings_test['rating'], pred_useruserTNS)

2.041387630879022

In [244]:
mean_absolute_error(df_ratings_test['rating'], pred_useruserTNS)

1.2289860275463496

In [245]:
nb_changed_values = df_ratings_test.shape[0]-pred_useruserTNS.count(average_train)
nb_changed_values

269

Remark:

In [227]:
len([a for a in nx.isolates(G_users)])

1812

We have 1812 users without any friend. That's why the prediction using the unipartite graph is not really performant.

## Rating prediction based on Multi Graph

We will use our different similarity matrices in order to develop a better recommendation system based on our two users similarity matrices.

In [246]:
from sklearn.preprocessing import MinMaxScaler

In [256]:
df_1 = pd.read_csv('similarities_bipartite_users.csv')
df_2 = pd.read_csv('similarities_unipartite_TNS_users.csv')
df_1 = df_1.set_index('Unnamed: 0')
df_2 = df_2.set_index('Unnamed: 0')

In [257]:
#Normalization of our similarity matrices
ar_1 = df_1.values
std1 = ar_1.std(ddof=1)
mean1 = ar_1.mean()
ar_1 = (ar_1-mean1)/std1
df_sim_1 = pd.DataFrame(ar_1, columns = users_list, index = users_list)

scaler = MinMaxScaler()
df_sim_1_sc = pd.DataFrame(scaler.fit_transform(df_sim_1), columns=df_sim_1.columns, index=df_sim_1.index)


In [258]:
ar_2 = df_2.values
std2 = ar_2.std(ddof=1)
mean2 = ar_2.mean()
ar_2 = (ar_2-mean2)/std2
df_sim_2 = pd.DataFrame(ar_2, columns = users_list, index = users_list)

scaler = MinMaxScaler()
df_sim_2_sc = pd.DataFrame(scaler.fit_transform(df_sim_2), columns=df_sim_2.columns, index=df_sim_2.index)


In [334]:
def rating_u_i(u,i,G_us_re,G_users,df_sim_1_sc,df_sim_2_sc,average_train):
    A = (G_users.degree(u)*G_users.number_of_nodes())/nx.adjacency_matrix(G_users).count_nonzero()
    R = (G_us_re.degree(u)*G_users.number_of_nodes())/G_us_re.number_of_edges()
    if A+R !=0:
        r = R/(A+R)
    else:
        r=1/2
    similarities_1 = np.asarray(df_sim_1_sc[u])
    similarities_2 = np.asarray(df_sim_2_sc[u])
    ar_sim = r*similarities_1 + (1-r)*similarities_2
    
    n,d = 0,0
    pred = 0
    for v in users_list:
        index_v = users_list.index(v)
        rating_v_i = G_us_re.get_edge_data(v,i,default={'weight': 0.0})['weight']
        if rating_v_i != 0:
            n += ar_sim[index_v]*rating_v_i
            d += ar_sim[index_v]
    if d !=0:
        pred = n/d
    else:
        pred = average_train
    return pred
    

In [335]:
def rating_predictions(df_ratings_test,G_us_re,G_users,df_sim_1_sc,df_sim_2_sc,average_train):
    pred = []
    for z in range(df_ratings_test.shape[0]):
        userid = df_ratings_test['user_id'][z]
        businessid = df_ratings_test['business_id'][z]
        pred.append(rating_u_i(userid,businessid,G_us_re,G_users,df_sim_1_sc,df_sim_2_sc,average_train))
    return pred

In [336]:
pred_multi = rating_predictions(df_ratings_test,G_us_re,G_users,df_sim_1_sc,df_sim_2_sc,average_train)


In [338]:
mean_squared_error(df_ratings_test['rating'], pred_multi)

1.9435698948439872

In [339]:
mean_absolute_error(df_ratings_test['rating'], pred_multi)

1.1687180289592196

In [340]:
nb_changed_values = df_ratings_test.shape[0]-pred_multi.count(average_train)
nb_changed_values

1592

In [459]:
df_ratings_test_pred = df_ratings_test[['user_id','business_id','rating']]
df_ratings_test_pred['predicted'] = pred_multi

In [460]:
df_ratings_test_pred

,user_id,business_id,rating,predicted
0,6X0i-oGUbh5DZdTHzFuKfg,u8C8pRvaHXg3PgDrsUHJHQ,5.0,3.779700
1,6X0i-oGUbh5DZdTHzFuKfg,dHkbBWmXXjaO_-9BgQyEPg,1.0,3.617508
2,iQt3ya8qaVJ347rJi5jSmA,dIZcPB3CtNjMn4O_p8QFxw,4.0,4.087520
3,iQt3ya8qaVJ347rJi5jSmA,i_t8WTwztuHweRqQ89hmuQ,4.0,3.439996
4,iQt3ya8qaVJ347rJi5jSmA,f_CDR6H4QL1K3SeaBe7r3g,3.0,3.540818
...,...,...,...,...
3929,VO42jNnadblgtODxPYVDRA,k6m3Msok7bto6biv5guEzg,2.0,3.617508
3930,nlHtklaFE5gfKCd6K-jeuQ,KGpsB2dsdkxl8SGVUlJbZw,1.0,3.617508
3931,1LtJ5w7YaxaN0equ3Oq6pw,dIUHCuiAlzkxfgCEOtky8w,5.0,3.617508
3932,EeOcpXst4ihchMBvNkAoUA,dIUHCuiAlzkxfgCEOtky8w,5.0,3.617508


## Rating prediction based on Multi Graph: Improvement

There are too many ratings that are not predicted (set to average_train).

In [408]:
average_users = []
for u in users_list:
    avg = df_ratings_train.loc[df_ratings_train['user_id']==u].rating.mean()
    if math.isnan(avg):
        average_users.append(average_train)
    else:
        average_users.append(avg)


In [468]:
average_users.count(average_train)

353

In [448]:
def rating_u_i_imp(u,i,G_us_re,G_users,df_sim_1_sc,df_sim_2_sc,average_users):
    A = (G_users.degree(u)*G_users.number_of_nodes())/nx.adjacency_matrix(G_users).count_nonzero()
    R = (G_us_re.degree(u)*G_users.number_of_nodes())/G_us_re.number_of_edges()
    if A+R !=0:
        r = R/(A+R)
    else:
        r=1/2
    similarities_1 = np.asarray(df_sim_1_sc[u])
    similarities_2 = np.asarray(df_sim_2_sc[u])
    ar_sim = r*similarities_1 + (1-r)*similarities_2
    
    index_u = users_list.index(u)
    avg_u = average_users[index_u]
    
    n,d = 0,0
    pred = avg_u
    for v in users_list:
        index_v = users_list.index(v)
        avg_v = average_users[index_v]
        rating_v_i = G_us_re.get_edge_data(v,i,default={'weight': 0.0})['weight']
        if rating_v_i != 0:
            n += ar_sim[index_v]*abs(rating_v_i - avg_v)
            d += ar_sim[index_v]
    if d !=0:
        pred += n/d
    return pred
    

In [449]:
def rating_predictions_imp(df_ratings_test,G_us_re,G_users,df_sim_1_sc,df_sim_2_sc,average_users):
    pred = []
    for z in range(df_ratings_test.shape[0]):
        userid = df_ratings_test['user_id'][z]
        businessid = df_ratings_test['business_id'][z]
        pred.append(rating_u_i_imp(userid,businessid,G_us_re,G_users,df_sim_1_sc,df_sim_2_sc,average_users))
    return pred

In [450]:
pred_multi_imp = rating_predictions_imp(df_ratings_test,G_us_re,G_users,df_sim_1_sc,df_sim_2_sc,average_users)


In [451]:
mean_squared_error(df_ratings_test['rating'], pred_multi_imp)

2.5195779609591726

In [452]:
mean_absolute_error(df_ratings_test['rating'], pred_multi_imp)

1.267716091784767

In [456]:
nb_changed_values = df_ratings_test.shape[0]-pred_multi_imp.count(average_train)
nb_changed_values

2149

In [461]:
df_ratings_test_pred_2 = df_ratings_test[['user_id','business_id','rating']]
df_ratings_test_pred_2['predicted'] = pred_multi_imp

In [462]:
df_ratings_test_pred_2

,user_id,business_id,rating,predicted
0,6X0i-oGUbh5DZdTHzFuKfg,u8C8pRvaHXg3PgDrsUHJHQ,5.0,3.618027
1,6X0i-oGUbh5DZdTHzFuKfg,dHkbBWmXXjaO_-9BgQyEPg,1.0,2.714286
2,iQt3ya8qaVJ347rJi5jSmA,dIZcPB3CtNjMn4O_p8QFxw,4.0,4.450527
3,iQt3ya8qaVJ347rJi5jSmA,i_t8WTwztuHweRqQ89hmuQ,4.0,4.433910
4,iQt3ya8qaVJ347rJi5jSmA,f_CDR6H4QL1K3SeaBe7r3g,3.0,4.764313
...,...,...,...,...
3929,VO42jNnadblgtODxPYVDRA,k6m3Msok7bto6biv5guEzg,2.0,3.617508
3930,nlHtklaFE5gfKCd6K-jeuQ,KGpsB2dsdkxl8SGVUlJbZw,1.0,3.617508
3931,1LtJ5w7YaxaN0equ3Oq6pw,dIUHCuiAlzkxfgCEOtky8w,5.0,3.617508
3932,EeOcpXst4ihchMBvNkAoUA,dIUHCuiAlzkxfgCEOtky8w,5.0,3.617508


In [464]:
df_ratings_test_pred_2[df_ratings_test_pred_2['predicted']==df_ratings_test_pred_2['rating']]


,user_id,business_id,rating,predicted
289,UeB-uvIiEcbL4zFG6ennOw,ZvSy8yaly9Aeh64Qm4Dsfw,3.0,3.0
292,UeB-uvIiEcbL4zFG6ennOw,lG9YdIb-87LK21XnMKvzFg,3.0,3.0
318,kYljKELrbyJgo8mxCCabMw,ECKXHrl2GqNKRMG9jp4trg,5.0,5.0
559,I9HeogVQnozJ1A-PgFaWJg,0eTpZzdjkPdz8HBxlljdGw,1.0,1.0
746,RW5fE95EYcqNrIqqwYzNXw,ZcR6MePiNSwL7DIHS6hGqQ,5.0,5.0
...,...,...,...,...
3791,VGQn1SDb1ogLmlSFj2u2rA,Hx-um-V0sTHjMZX0UcW0kA,5.0,5.0
3807,CC8_I1VWGK4k12skvoTBug,CObgcxFJuEXZZpm4Mw3z1A,1.0,1.0
3821,9YnlUM2uzxdbiq655JSEww,dIUHCuiAlzkxfgCEOtky8w,5.0,5.0
3888,pc7xSPuxArX_FaNfvDkqzQ,RAJUQvRLopFmIklzqaUYeA,4.0,4.0


In [469]:
#df_ratings_test_pred[df_ratings_test_pred['rating']==df_ratings_test_pred['predicted']]
df_ratings_test_pred_2[df_ratings_test_pred_2['predicted']==average_train]

,user_id,business_id,rating,predicted
153,SaKMmwfg0p35kp8bLgvqlQ,VHsNB3pdGVcRgs6C3jt6Zg,1.0,3.617508
154,SaKMmwfg0p35kp8bLgvqlQ,B9W60PTxn7XOZfk4BiTvyw,5.0,3.617508
155,SaKMmwfg0p35kp8bLgvqlQ,3G_bblT7MQIlXx8BBorILQ,2.0,3.617508
161,w5xsTd0rUnTp9wd0w1qTpg,Ah4i15g8Ow_zphzcpulTxQ,4.0,3.617508
162,w5xsTd0rUnTp9wd0w1qTpg,9MnbQg7kfb_WgxoV0hXKSQ,5.0,3.617508
...,...,...,...,...
3929,VO42jNnadblgtODxPYVDRA,k6m3Msok7bto6biv5guEzg,2.0,3.617508
3930,nlHtklaFE5gfKCd6K-jeuQ,KGpsB2dsdkxl8SGVUlJbZw,1.0,3.617508
3931,1LtJ5w7YaxaN0equ3Oq6pw,dIUHCuiAlzkxfgCEOtky8w,5.0,3.617508
3932,EeOcpXst4ihchMBvNkAoUA,dIUHCuiAlzkxfgCEOtky8w,5.0,3.617508


In [375]:
df_ratings_test_pred_itemCF = df_ratings_test[['user_id','business_id','rating']]
df_ratings_test_pred_itemCF['predicted'] = pred_itemCF
df_ratings_test_pred_itemCF[df_ratings_test_pred_itemCF['predicted']==average_train]

,user_id,business_id,rating,predicted
1,6X0i-oGUbh5DZdTHzFuKfg,dHkbBWmXXjaO_-9BgQyEPg,1.0,3.617508
15,ZdeK97L0l1Z4Uv1vnBtlXg,XiJG_I82LssFbGDIMsHYvQ,3.0,3.617508
16,ZdeK97L0l1Z4Uv1vnBtlXg,sjVVGhnLJrcEq0ErUL168w,3.0,3.617508
19,ZdeK97L0l1Z4Uv1vnBtlXg,1tzuSAcAFy-E7UbEhPjBjg,4.0,3.617508
26,ZdeK97L0l1Z4Uv1vnBtlXg,RucrUwlX6hvGlUOrakViYw,5.0,3.617508
...,...,...,...,...
3929,VO42jNnadblgtODxPYVDRA,k6m3Msok7bto6biv5guEzg,2.0,3.617508
3930,nlHtklaFE5gfKCd6K-jeuQ,KGpsB2dsdkxl8SGVUlJbZw,1.0,3.617508
3931,1LtJ5w7YaxaN0equ3Oq6pw,dIUHCuiAlzkxfgCEOtky8w,5.0,3.617508
3932,EeOcpXst4ihchMBvNkAoUA,dIUHCuiAlzkxfgCEOtky8w,5.0,3.617508
